In [3]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import random
from pathlib import Path
import pandas as pd
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import math
import torch.optim as optim
import torch.nn as nn
from torchvision.models import resnet18, efficientnet_b3
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import onnxruntime as ort

/usr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1748653007.468353     463 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748653007.475299     463 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
torch.cuda.is_available()

True

In [5]:
import kagglehub
#dataset_path = kagglehub.dataset_download("jessicali9530/lfw-dataset")
dataset_path = kagglehub.dataset_download("chiragsaipanuganti/morph")

In [34]:
dataset_path

'/root/.cache/kagglehub/datasets/chiragsaipanuganti/morph/versions/2'

In [6]:
image_paths = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.jpg')):
            image_paths.append(os.path.join(root, file))

In [36]:
def populate_whitecard(image_paths, naming, height=1540, width=1754, im_width=250, im_height=250, spacing=80):
    whitecard = np.ones((height, width, 3), dtype=np.uint8) * 255
    x = 10
    y = 10

    diagonal_size = int(np.sqrt(im_width**2 + im_height**2)) + 10

    for i, name in zip(image_paths, naming):
        img = cv2.imread(i)
        img = cv2.resize(img, (im_width, im_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        rot_canvas = np.ones((diagonal_size, diagonal_size, 3), dtype=np.uint8) * 255
        start_x = (diagonal_size - im_width) // 2
        start_y = (diagonal_size - im_height) // 2
        rot_canvas[start_y:start_y + im_height, start_x:start_x + im_width] = img

        rot_angle = random.uniform(-180, 180)
        center = (diagonal_size // 2, diagonal_size // 2)
        rot_matrix = cv2.getRotationMatrix2D(center, rot_angle, 1.0)
        rot_img = cv2.warpAffine(rot_canvas, rot_matrix, (diagonal_size, diagonal_size), borderValue=(255, 255, 255))

        whitecard[y:y+diagonal_size, x:x+diagonal_size] = rot_img

        cv2.putText(whitecard, str(name), (x + diagonal_size // 2, y + diagonal_size + (spacing // 2)), 
                   cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1.3, (0, 0, 0), 2, cv2.LINE_AA)

        x += diagonal_size + 40
        if x > (width-diagonal_size):
            x = 10
            y += diagonal_size + spacing
        if y > (height-diagonal_size):
            return whitecard
    return whitecard

def get_cards(image_paths, num, im_width, im_height):
    cards = []
    random.shuffle(image_paths)
    for i in range(num):
        card = populate_whitecard(image_paths[i*20:(i+1)*20], [f"{i*20+j+1}" for j in range(20)], im_height=im_height, im_width=im_width)
        cards.append(card)
    return cards

In [40]:
cards = get_cards(image_paths, 10, im_width=200, im_height=200)

output_path = Path("cards")

output_path.mkdir(exist_ok=True)

for i, card in enumerate(cards):
    card_bgr = cv2.cvtColor(card, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'cards/card_{i+1}.jpg', card_bgr)

In [4]:
def populate_whitecard_labels_v2(image_paths, naming, height=1540, width=1754, im_width=250, im_height=250, spacing=80):
    whitecard = np.ones((height, width, 3), dtype=np.uint8) * 255
    x = 10
    y = 10

    diagonal_size = int(np.sqrt(im_width**2 + im_height**2)) + 10
    label_entries = []

    for i, name in zip(image_paths, naming):
        img = cv2.imread(i)
        img = cv2.resize(img, (im_width, im_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        rot_canvas = np.ones((diagonal_size, diagonal_size, 3), dtype=np.uint8) * 255
        start_x = (diagonal_size - im_width) // 2
        start_y = (diagonal_size - im_height) // 2
        rot_canvas[start_y:start_y + im_height, start_x:start_x + im_width] = img

        rot_angle = random.uniform(-180, 180)
        center = (diagonal_size // 2, diagonal_size // 2)
        rot_matrix = cv2.getRotationMatrix2D(center, rot_angle, 1.0)
        rot_img = cv2.warpAffine(rot_canvas, rot_matrix, (diagonal_size, diagonal_size), borderValue=(255, 255, 255))

        whitecard[y:y+diagonal_size, x:x+diagonal_size] = rot_img

        cv2.putText(whitecard, str(name), (x + diagonal_size // 2, y + diagonal_size + (spacing // 2)), 
                   cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1.3, (0, 0, 0), 2, cv2.LINE_AA)
        
        photo_x_center = (x + diagonal_size / 2) / width
        photo_y_center = (y + diagonal_size / 2) / height
        photo_width = diagonal_size / width
        photo_height = (diagonal_size + spacing) / height
        label_entries.append(f"0 {photo_x_center:.6f} {photo_y_center:.6f} {photo_width:.6f} {photo_height:.6f}")

        id_x_center = photo_x_center
        id_y_center = (y + diagonal_size + spacing / 2) / height
        id_width = photo_width
        id_height = spacing / height
        label_entries.append(f"1 {id_x_center:.6f} {id_y_center:.6f} {id_width:.6f} {id_height:.6f}")

        x += diagonal_size + 40
        if x > (width-diagonal_size):
            x = 10
            y += diagonal_size + spacing
        if y > (height-diagonal_size):
            break
    return whitecard, label_entries

def get_cards_labels(image_paths, num, im_width, im_height, output_path, starting_idx=0):
    images_path = Path("images") / Path(output_path)
    images_path.mkdir(parents=True, exist_ok=True)
    labels_path = Path("labels") / Path(output_path)
    labels_path.mkdir(parents=True, exist_ok=True)
    cards = []

    random.shuffle(image_paths)
    for i in range(starting_idx, starting_idx+num):
        imgs = image_paths[i*50:(i+1)*50]
        names = [f"{i*50+j+1}" for j in range(len(imgs))]
        card, labels = populate_whitecard_labels_v2(imgs, names, im_width=im_width, im_height=im_height)
        filename = f"card_{i+1}"
        
        card_bgr = cv2.cvtColor(card, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(images_path / f"{filename}.jpg"), card_bgr)

        with open(labels_path / f"{filename}.txt", "w") as f:
            f.write("\n".join(labels))
    return cards

In [5]:
for i in ['train_v2', 'valid_v2', 'test_v2']:
    get_cards_labels(image_paths, num=30, im_width=250, im_height=250, output_path=i, starting_idx=0)
    get_cards_labels(image_paths, num=30, im_width=200, im_height=200, output_path=i, starting_idx=30)
    get_cards_labels(image_paths, num=30, im_width=150, im_height=150, output_path=i, starting_idx=60)
    get_cards_labels(image_paths, num=15, im_width=175, im_height=175, output_path=i, starting_idx=90)
    get_cards_labels(image_paths, num=15, im_width=275, im_height=275, output_path=i, starting_idx=105)

In [23]:
def populate_card_labels_v3(image_path, height=400, width=400, im_width=250, im_height=250):
	card = np.ones((height, width, 3), dtype=np.uint8) * 255
	img = cv2.imread(image_path)
	img = cv2.resize(img, (im_width, im_height))
	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

	diagonal_size = int(np.sqrt(im_width**2 + im_height**2)) + 10
	rot_canvas = np.ones((diagonal_size, diagonal_size, 3), dtype=np.uint8) * 255
	start_x = (diagonal_size - im_width) // 2
	start_y = (diagonal_size - im_height) // 2
	rot_canvas[start_y:start_y + im_height, start_x:start_x + im_width] = img

	rot_angle = random.uniform(-179, 180)
	center = (diagonal_size // 2, diagonal_size // 2)
	rot_matrix = cv2.getRotationMatrix2D(center, rot_angle, 1.0)
	rot_img = cv2.warpAffine(rot_canvas, rot_matrix, (diagonal_size, diagonal_size), borderValue=(255, 255, 255))

	card_x = (width - diagonal_size) // 2
	card_y = (height - diagonal_size) // 2
	card[card_y:card_y+diagonal_size, card_x:card_x+diagonal_size] = rot_img

	corners = np.array([
		[start_x, start_y],
		[start_x + im_width, start_y],
		[start_x + im_width, start_y + im_height],
		[start_x, start_y + im_height]
	], dtype=np.float32)

	ones = np.ones((corners.shape[0], 1))
	corners_homo = np.hstack([corners, ones])
	rotated_corners = (rot_matrix @ corners_homo.T).T

	x_min, y_min = rotated_corners.min(axis=0)
	x_max, y_max = rotated_corners.max(axis=0)

	bbox_cx = (x_min + x_max) / 2 / diagonal_size
	bbox_cy = (y_min + y_max) / 2 / diagonal_size
	bbox_w = (x_max - x_min) / diagonal_size
	bbox_h = (y_max - y_min) / diagonal_size

	norm_corners = []
	for x, y in rotated_corners:
		norm_corners.extend([x / diagonal_size, y / diagonal_size, 2])

	label = [0, bbox_cx, bbox_cy, bbox_w, bbox_h] + norm_corners

	return card, label

def get_cards_labels(image_paths, num, im_width, im_height, output_path, starting_idx=0):
	output_path = Path(output_path)
	images_path = "images" / output_path
	labels_path = "labels" / output_path
	images_path.mkdir(parents=True, exist_ok=True)
	labels_path.mkdir(parents=True, exist_ok=True)
	cards = []

	random.shuffle(image_paths)
    
	for i in range(starting_idx, starting_idx+num):
		card, label = populate_card_labels_v3(
			image_paths[i],
			im_width=im_width,
			im_height=im_height
		)

		filename = f"card_{i+1}"
		image_file = images_path / f"{filename}.jpg"
		label_file = labels_path / f"{filename}.txt"
        
		card_bgr = cv2.cvtColor(card, cv2.COLOR_RGB2BGR)
		cv2.imwrite(str(image_file), card_bgr)
        
		label_line = " ".join(f"{v:.6f}" if isinstance(v, float) else str(v) for v in label)

		with open(label_file, "w") as f:
			f.write(label_line)

		cards.append({"filename": f"{filename}.jpg", "label": label})
    
	return cards

In [24]:
for i in ['train_v3', 'valid_v3', 'test_v3']:
	for j, k in zip([250, 200, 150, 175, 275], [0, 500, 1000, 1500, 2000]):
		cards = get_cards_labels(image_paths, num=500, im_width=j, im_height=j, output_path=i, starting_idx=k)

In [11]:
model = YOLO('yolo11n.pt')  # nano
model = YOLO('yolo11s.pt')  # small
model = YOLO('yolo11m.pt')  # medium
model = YOLO('yolo11l.pt')  # large
model = YOLO('yolo11x.pt')  # extra large

In [9]:
#model = YOLO('yolo11n-pose.pt')  # nano
#model = YOLO('yolo11s-pose.pt')  # small
#model = YOLO('yolo11m-pose.pt')  # medium
#model = YOLO('yolo11l-pose.pt')  # large
model = YOLO('yolo11x-pose.pt')  # extra large

100%|██████████| 113M/113M [00:11<00:00, 10.4MB/s] 


In [30]:
from ultralytics import YOLO

model = YOLO('yolo11x.pt')

model.train(
    data='person_entry.yaml',
    epochs=100,
    imgsz=1024,
    batch=8,
    name='person_entry_model'
)

New https://pypi.org/project/ultralytics/8.3.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=person_entry.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=person_entry_model5, nbs=64, nms=False

train: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/train... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 1081.81it/s]

train: New cache created: /repos/uni-si/R2S4/DNN/End_project/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1304.3±1551.3 MB/s, size: 364.4 KB)


val: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/valid... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 982.55it/s]

...

                   all        120       2040          1          1      0.995      0.995



100 epochs completed in 3.998 hours.
Optimizer stripped from runs/detect/person_entry_model5/weights/last.pt, 114.5MB
Optimizer stripped from runs/detect/person_entry_model5/weights/best.pt, 114.5MB

Validating runs/detect/person_entry_model5/weights/best.pt...
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:10<00:00,  1.26s/it]


                   all        120       2040          1          1      0.995      0.995
Speed: 0.7ms preprocess, 68.5ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/person_entry_model5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x76980db69450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [12]:
from ultralytics import YOLO

# Załaduj wytrenowany model
model = YOLO('runs/detect/person_entry_model5/weights/best.pt')

# Uruchom inferencję na folderze testowym
results = model.predict(
    source='images/test',
    save=True,
    conf=0.25,
    imgsz=1024
)


image 1/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_1.jpg: 928x1024 12 person_entrys, 36.9ms
image 2/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_10.jpg: 928x1024 12 person_entrys, 27.4ms
image 3/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_100.jpg: 928x1024 12 person_entrys, 27.6ms
image 4/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_101.jpg: 928x1024 12 person_entrys, 27.1ms
image 5/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_102.jpg: 928x1024 12 person_entrys, 26.9ms
image 6/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_103.jpg: 928x1024 12 person_entrys, 25.7ms
image 7/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_104.jpg: 928x1024 12 person_entrys, 24.7ms
image 8/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_105.jpg: 928x1024 12 person_entrys, 24.6ms
image 9/120 /repos/uni-si/R2S4/DNN/End_project/images/test/card_106.jpg: 928x1024 12 person_entrys, 38.0ms
image 10/120 /repos/uni-si/R2S4/DNN/End

In [13]:
boxes = results[3].boxes.xywhn.cpu().numpy()
classes = results[3].boxes.cls.cpu().numpy().astype(int)

display(boxes)
display(classes)
display(results[3].path)

array([[    0.56888,     0.72514,     0.20684,     0.28799],
       [    0.56767,     0.43779,     0.20654,     0.28667],
       [    0.79746,     0.14979,     0.20588,     0.28855],
       [    0.10948,     0.15001,      0.2072,     0.28746],
       [    0.56791,      0.1501,     0.20692,     0.28779],
       [    0.79792,     0.72557,     0.20739,     0.28835],
       [    0.33948,     0.72529,     0.20683,     0.28873],
       [    0.10942,     0.72521,     0.20606,     0.28853],
       [    0.33902,     0.43795,     0.20658,     0.28731],
       [    0.79866,     0.43754,      0.2068,     0.28784],
       [    0.10886,     0.43753,     0.20616,       0.287],
       [    0.33956,     0.14986,     0.20596,     0.28766]], dtype=float32)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

'/repos/uni-si/R2S4/DNN/End_project/images/test/card_101.jpg'

In [6]:
def prepare_yolo2_training_data(cards_path, yolo1_results, output_img_dir, output_lbl_dir):
    cards_dir = Path(cards_path)
    Path(output_img_dir).mkdir(parents=True, exist_ok=True)
    Path(output_lbl_dir).mkdir(parents=True, exist_ok=True)

    # Mapowanie nazwy pliku na pełną ścieżkę
    card_paths = {p.name: p for p in cards_dir.glob("*.jpg")}

    for result in yolo1_results:
        img_path = Path(result.path)
        img_name = img_path.name

        if img_name not in card_paths:
            print(f"Warning: {img_name} not found in cards_dir.")
            continue

        # Wyciągnięcie ID karty z nazwy pliku
        img_id = int(img_name.split("_")[1].split(".")[0])

        # Dobranie odpowiedniego ratio na podstawie img_id
        if img_id <= 30:
            photo_height_ratio = 0.82
        elif img_id <= 60:
            photo_height_ratio = 0.80
        elif img_id <= 90:
            photo_height_ratio = 0.75
        elif img_id <= 105:
            photo_height_ratio = 0.77
        else:
            photo_height_ratio = 0.842

        id_height_ratio = 1.0 - photo_height_ratio

        img = cv2.imread(str(card_paths[img_name]))
        h, w = img.shape[:2]

        boxes = result.boxes.xywhn.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy().astype(int)

        for idx, (box, cls) in enumerate(zip(boxes, classes)):
            x_c, y_c, bw, bh = box

            # Przeliczenie boxa YOLO na piksele
            x_min = int((x_c - bw / 2) * w)
            x_max = int((x_c + bw / 2) * w)
            y_min = int((y_c - bh / 2) * h)
            y_max = int((y_c + bh / 2) * h)

            # Wycinamy crop z oryginalnego obrazu
            crop = img[y_min:y_max, x_min:x_max]

            # Zapis cropa do output_img_dir
            crop_filename = f"{img_name[:-4]}_entry_{idx}.jpg"
            cv2.imwrite(str(Path(output_img_dir) / crop_filename), crop)

            # Tworzymy label dla drugiego modelu
            labels = []

            # Box dla zdjęcia (klasa 0)
            px_c = 0.5
            py_c = photo_height_ratio / 2
            pw = 1.0
            ph = photo_height_ratio
            labels.append(f"0 {px_c:.6f} {py_c:.6f} {pw:.6f} {ph:.6f}")

            # Box dla ID (klasa 1)
            ix_c = 0.5
            iy_c = photo_height_ratio + id_height_ratio / 2
            iw = 1.0
            ih = id_height_ratio
            labels.append(f"1 {ix_c:.6f} {iy_c:.6f} {iw:.6f} {ih:.6f}")

            label_filename = f"{img_name[:-4]}_entry_{idx}.txt"
            with open(Path(output_lbl_dir) / label_filename, "w") as f:
                f.write("\n".join(labels))


In [7]:
from ultralytics import YOLO
model = YOLO('runs/detect/person_entry_model5/weights/best.pt')

for i in ['train_v2', 'valid_v2', 'test_v2']:
    results = model.predict(
        source=f'images/{i}',
        save=False,
        conf=0.25,
        imgsz=1024
    )

    prepare_yolo2_training_data(f'images/{i}', results, output_img_dir=f'images/{i}_yolo2', output_lbl_dir=f'labels/{i}_yolo2')


image 1/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_1.jpg: 928x1024 12 person_entrys, 33.7ms
image 2/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_10.jpg: 928x1024 12 person_entrys, 25.0ms
image 3/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_100.jpg: 928x1024 24 person_entrys, 24.9ms
image 4/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_101.jpg: 928x1024 24 person_entrys, 25.5ms
image 5/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_102.jpg: 928x1024 24 person_entrys, 25.6ms
image 6/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_103.jpg: 928x1024 24 person_entrys, 25.0ms
image 7/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_104.jpg: 928x1024 24 person_entrys, 25.2ms
image 8/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_105.jpg: 928x1024 24 person_entrys, 24.5ms
image 9/120 /repos/uni-si/R2S4/DNN/End_project/images/train_v2/card_106.jpg: 928x1024 12 person_entrys, 25.0ms
ima

In [8]:
from ultralytics import YOLO

model = YOLO('yolo11x.pt')

model.train(
    data='split_image_id.yaml',
    epochs=100,
    imgsz=320,
    batch=8,
    name='split_image_id_model'
)

New https://pypi.org/project/ultralytics/8.3.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=split_image_id.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=split_image_id_model, nbs=64, nms=Fal

train: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/train_v2_yolo2... 2400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2400/2400 [00:02<00:00, 1053.34it/s]

train: New cache created: /repos/uni-si/R2S4/DNN/End_project/labels/train_v2_yolo2.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 515.0±350.9 MB/s, size: 16.5 KB)


val: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/valid_v2_yolo2... 2400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2400/2400 [00:02<00:00, 1027.01it/s]

...

                   all       2400       4800          1          1      0.995      0.995



100 epochs completed in 1.400 hours.
Optimizer stripped from runs/detect/split_image_id_model/weights/last.pt, 114.3MB
Optimizer stripped from runs/detect/split_image_id_model/weights/best.pt, 114.3MB

Validating runs/detect/split_image_id_model/weights/best.pt...
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11x summary (fused): 190 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:13<00:00, 11.27it/s]


                   all       2400       4800          1          1      0.995      0.995
                 image       2400       2400          1          1      0.995      0.995
                    id       2400       2400          1          1      0.995      0.995
Speed: 0.0ms preprocess, 1.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/split_image_id_model


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7446dccb0610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [21]:
from ultralytics import YOLO

# Załaduj wytrenowany model
model = YOLO('runs/detect/split_image_id_model/weights/best.pt')

# Uruchom inferencję na folderze testowym
results = model.predict(
    source='images/test_v2_yolo2/card_1_entry_0.jpg',
    save=True,
    conf=0.25,
    imgsz=320
)


image 1/1 /repos/uni-si/R2S4/DNN/End_project/images/test_v2_yolo2/card_1_entry_0.jpg: 320x288 1 image, 1 id, 14.2ms
Speed: 0.6ms preprocess, 14.2ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 288)
Results saved to runs/detect/predict4


In [25]:
from ultralytics import YOLO

model = YOLO('yolo11x-pose.pt')

model.train(
    data='image_rotation.yaml',
    epochs=100,
    imgsz=320,
    batch=8,
    name='image_rotation_model'
)

New https://pypi.org/project/ultralytics/8.3.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=image_rotation.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=image_rotation_model, nbs=64, nm

train: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/train_v3... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:02<00:00, 1039.98it/s]

train: New cache created: /repos/uni-si/R2S4/DNN/End_project/labels/train_v3.cache


WARNING ⚠️ No 'flip_idx' array defined in data.yaml, setting augmentation 'fliplr=0.0'
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 519.3±298.4 MB/s, size: 25.0 KB)


val: Scanning /repos/uni-si/R2S4/DNN/End_project/labels/valid_v3... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:02<00:00, 993.89it/s] 


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


      8/100      5.12G     0.9371      1.615     0.4613     0.6418      1.309         15        320: 100%|██████████| 313/313 [00:39<00:00,  7.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:16<00:00,  9.26it/s]


                   all       2500       2500          1          1      0.995      0.797      0.987      0.987      0.976      0.779

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


      9/100      5.41G     0.9398      1.581     0.4538      0.637      1.312         18        320:  57%|█████▋    | 179/313 [00:25<00:19,  7.04it/s]


KeyboardInterrupt: 

In [22]:
from ultralytics import YOLO

# Załaduj wytrenowany model
model = YOLO('runs/pose/image_rotation_model4/weights/best.pt')

# Uruchom inferencję na folderze testowym
results = model.predict(
    source='images/test_v3',
    save=True,
    conf=0.25,
    imgsz=320
)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2500 /repos/uni-si/R2S4/DNN/End_project/images/test_v3/card_1.jpg: 320x320 1 rotated_photo, 46.7ms
image 2/2500 /repos/uni-si/R2S4/DNN/End_project/images/test_v3/card_10.jpg: 320x320 1 rotated_photo, 47.9ms
image 3/2500 /repos/uni-si/R2S4/DNN/End_project/images/test_v3/card_100.jpg: 320x320 1 rotated_photo, 45.5ms
image 4/2500 /repos/uni-si/R2S4/DNN/End_project/images/test_v3/card_1000.jpg: 320x320 1 rotated_photo, 14.5ms
image 5/2500 /repos/un

In [1]:
import onnxruntime as ort
import numpy as np
import cv2

# Wczytaj model
session = ort.InferenceSession("deep-image-orientation-angle-detection.onnx")

# Wczytaj i przetwórz obraz
image = cv2.imread("cards_image/card_7500.jpg")
image_resized = cv2.resize(image, (224, 224))
image_normalized = image_resized.astype(np.float32) / 255.0
input_tensor = np.expand_dims(image_normalized.transpose(2, 0, 1), axis=0)

# Inferencja
outputs = session.run(None, {"image": input_tensor})
angle = outputs[0][0][0]
print(f"Przewidywany kąt: {angle:.2f}°")


Przewidywany kąt: -47.40°


In [8]:
(h, w) = image.shape[:2]
center = (w // 2, h // 2)

M = cv2.getRotationMatrix2D(center, 48.05215817737758, 1.0)
rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
cv2.imwrite("rotated_image.jpg", rotated)

True

In [26]:
boxes = results[0].boxes.xywhn.cpu().numpy()
classes = results[0].boxes.cls.cpu().numpy().astype(int)

display(boxes)
display(classes)
display(results[0].path)

array([[    0.50038,     0.41057,     0.99924,     0.81992],
       [    0.50009,     0.90997,     0.99911,     0.18001]], dtype=float32)

array([0, 1])

'/repos/uni-si/R2S4/DNN/End_project/images/test_v2_yolo2/card_1_entry_0.jpg'

In [ ]:
class CardProcessor:
	def __init__(self):
		self.detection_model = YOLO('runs/detect/person_entry_model5/weights/best.pt')
		self.split_model = YOLO('runs/detect/split_image_id_model/weights/best.pt')
		self.rotation_model = ort.InferenceSession("deep-image-orientation-angle-detection.onnx")

		self.debug_ocr_path = Path("debug_ocr_images")
		self.debug_ocr_path.mkdir(exist_ok=True)
		self.ocr_debug_counter = 0

		try:
			self.trocr_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-stage1', use_fast=True)
			self.trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-stage1')
			if torch.cuda.is_available() and self.trocr_model is not None:
				self.trocr_model.to('cuda')
			print("TrOCR model loaded successfully.")
		except Exception as e:
			print(f"Error loading TrOCR model: {e}. TrOCR will not be available.")
			self.trocr_processor = None
			self.trocr_model = None

	def process_card(self, card_path):
		card = cv2.imread(card_path)

		pair_regions = self.detection_model.predict(source=card_path, save=False, conf=0.25, imgsz=1024)
		if not pair_regions or len(pair_regions) == 0:
			print(f"No regions detected in {card_path}.")
			return []

		print(f"Found {len(pair_regions)} image-id pair regions.")

		boxes = pair_regions[0].boxes.xywhn.cpu().numpy()

		results = []

		for i, box in enumerate(boxes):
			crop_box = self.extract_crop(card, box)

			split_regions = self.split_model.predict(source=crop_box, save=False, conf=0.25, imgsz=320)

			split_boxes = split_regions[0].boxes.xywhn.cpu().numpy()

			image_crop = self.extract_crop(crop_box, split_boxes[0])
			id_crop = self.extract_crop(crop_box, split_boxes[1])

			rotated_image, angle = self.correct_orientation(image_crop)

			id = self.extract_id(id_crop)

			results.append({
				'id': id,
				'rotated_crop': rotated_image,
				'angle': angle,
				'source_card': Path(card_path).name,
				'region_idx': i,
				'region': box
			})

		return results

	def extract_id(self, region):
		if self.trocr_model is None or self.trocr_processor is None:
			print("Debug - TrOCR model not initialized, skipping OCR.")
			return "OCR_ERROR"

		ocr_attempt_filename_prefix = f"ocr_input_{self.ocr_debug_counter}"
		self.ocr_debug_counter += 1

		try:
			pixel_values = self.trocr_processor(images=region, return_tensors="pt").pixel_values
			if torch.cuda.is_available():
				pixel_values = pixel_values.to(self.trocr_model.device)

			generated_ids = self.trocr_model.generate(pixel_values, max_length=10)

			decoded_texts = self.trocr_processor.batch_decode(generated_ids, skip_special_tokens=True)

			if not decoded_texts or not decoded_texts[0].strip():
				debug_filename = self.debug_ocr_path / f"{ocr_attempt_filename_prefix}_fails_empty.png"
				cv2.imwrite(str(debug_filename), region)
				print(f"Debug - Saved empty/whitespace OCR crop to {debug_filename}")
				return "ID_NOT_DETECTED"

			id_text_from_ocr = decoded_texts[0].strip()

			numeric_id_text = "".join(filter(str.isdigit, id_text_from_ocr))

			if not numeric_id_text:
				debug_filename = self.debug_ocr_path / f"{ocr_attempt_filename_prefix}_fails_nonumeric_{id_text_from_ocr.replace(' ','_')}.png"
				cv2.imwrite(str(debug_filename), region)
				print(f"Debug - Saved non-numeric OCR crop to {debug_filename} (Raw: '{id_text_from_ocr}')")
				return "ID_NOT_DETECTED"

			print(f"Debug - TrOCR extracted ID: {numeric_id_text}")
			return numeric_id_text

		except Exception as e:
			print(f"Debug - Exception during TrOCR processing in extract_id for region {region}: {e}")
			return "OCR_ERROR"

	def find_rotation_angle(self, image):
		image_resized = cv2.resize(image, (224, 224))
		image_normalized = image_resized.astype(np.float32) / 255.0
		input_tensor = np.expand_dims(image_normalized.transpose(2, 0, 1), axis=0)
		outputs = session.run(None, {"image": input_tensor})
		angle = outputs[0][0][0]
		return angle

	def correct_orientation(self, crop):
		angle = self.find_rotation_angle(crop)
		for i in range(3):
			angle = self.find_rotation_angle(crop)
			print(angle)
		rotated = self.rotate_image(crop, angle)

		return rotated, angle

	def evaluate_alignment_score(self, image):
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
		score = np.var(sobel)
		return score

	def rotate_image(self, image, angle):
		(h, w) = image.shape[:2]
		center = (w // 2, h // 2)

		M = cv2.getRotationMatrix2D(center, -angle, 1.0)
		rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
		return rotated

	def extract_crop(self, card, region):
		card_h, card_w = card.shape[:2]

		region_x, region_y, region_w, region_h = region
		x_min = int((region_x - region_w / 2) * card_w)
		x_max = int((region_x + region_w / 2) * card_w)
		y_min = int((region_y - region_h / 2) * card_h)
		y_max = int((region_y + region_h / 2) * card_h)

		crop = card[y_min:y_max, x_min:x_max]
		return crop


def process_cards(input_dir, output_dir):
	processor = CardProcessor()

	input_path = Path(input_dir)
	output_path = Path(output_dir)
	output_path.mkdir(exist_ok=True)

	all_images_info = []

	for image_file in input_path.glob('*.jpg'):
		print(f"Processing {image_file.name}...")
		results = processor.process_card(str(image_file))

		for result in results:
			id = result['id']
			if id in ["ID_NOT_FOUND", "ID_NOT_DETECTED", "OCR_ERROR"]:
				filename = f"unknown_{result['source_card']}_{result['region_idx']}.jpg"
			else:
				filename = f"{id}.jpg"

			output_file = output_path / filename

			counter = 1
			while output_file.exists():
				if id in ["ID_NOT_FOUND", "ID_NOT_DETECTED", "OCR_ERROR"]:
					filename = f"unknown_{result['source_card']}_{result['region_idx']}_{counter}.jpg"
				else:
					filename = f"{id}_{counter}.jpg"
				output_file = output_path / filename
				counter += 1

			cv2.imwrite(str(output_file), result['rotated_crop'])

			all_images_info.append({
				'id': result['id'],
				'filename': filename,
				'source_card': result['source_card'],
				'region_idx': result['region_idx'],
				'angle': result['angle'],
				'region_x': result['region'][0],
				'region_y': result['region'][1],
				'region_w': result['region'][2],
				'region_h': result['region'][3],
			})

			print(f"  Saved: {filename} (ID: {result['id']}, rotated: {result['angle']}°)")

	if all_images_info:
		df = pd.DataFrame(all_images_info)
		df.to_csv(output_path / 'images_info.csv', index=False)

		print(f"\nProcessing complete!")
		print(f"Total images saved: {len(all_images_info)}")
		print(f"All images saved to: {output_path}")
		print(f"Image info saved to: {output_path / 'all_images.csv'}")

	return all_images_info

In [28]:
results = process_cards('cards', 'extracted')

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 384,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_

TrOCR model loaded successfully.
Processing card_2.jpg...

image 1/1 /repos/uni-si/R2S4/DNN/End_project/cards/card_2.jpg: 928x1024 20 person_entrys, 25.8ms
Speed: 3.1ms preprocess, 25.8ms inference, 1.2ms postprocess per image at shape (1, 3, 928, 1024)
Found 1 image-id pair regions.

0: 320x256 1 image, 1 id, 14.7ms
Speed: 0.4ms preprocess, 14.7ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 256)
68.14432
68.14432
68.14432
Debug - TrOCR extracted ID: 29

0: 320x256 1 image, 1 id, 13.0ms
Speed: 0.9ms preprocess, 13.0ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 256)
38.985474
38.985474
38.985474
Debug - TrOCR extracted ID: 39

0: 320x256 1 image, 1 id, 12.8ms
Speed: 0.5ms preprocess, 12.8ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 256)
1.3808746
1.3808746
1.3808746
Debug - TrOCR extracted ID: 33

0: 320x256 1 image, 1 id, 12.1ms
Speed: 0.6ms preprocess, 12.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 256)
154.82233
1